In [ ]:
# Hubbard +U Correction Program for LK-99: Hubbard+UCorrection.ipynb

The program below applies density functional theory (DFT) calculations with Hubbard U corrections (DFT+U) on a pre-relaxed LK-99 structure, so that the electronic
properties of the material can be studied thoroughly. This code is particularly useful in evaluating the superconductivity of the LK-99 material. 

The results of Hubbard+UCorrection.ipynb contribute to the description of Cu-3d electrons, which proves to be relevant for determining the band structure near 
the Fermi level, for one. Flat bands may increase electron-phonon coupling or unconventional pairing. Additionally, such a description will aid in the 
identification of magnetic properties of LK-99, including $Cu^{2+}$ spin states might affect spin-fluctuation-mediated pairing. 

The program loads the relaxed LK-99 structure, reading the atomic position and lattice parameters from a particular output file that had been previously generated
in a prior structural relaxation with the PBEsol functional. 

The next step is the establishment of the Hubbard U parameters, where a +U correction (of $5.0 eV$) is applied to Cu-3d orbitals so that electron localization is
more seamlessly described. 

Generating the input for Quantum ESPRESSO follows from the previous step, which entails the use of pseudopotentials with the PBEsol exchange-correlation functional.
DFT+U is equipped, which determines Hubbard U/J parameters for Cu. Also, the computational parameters, such as the energy cut-oofs, k-point gri, and smearing for 
metallic systems, are set. 

Afterwards, the self-consistent field calculation (with U) is run via pw.x from QE. 

The program analyzes the results to ensure that each step was executed correctly. The total energy is then extracted.


### Background Theory

DFT+U fixes self-cinteraction errors in classic density functional theory, specifically for localized d/f-electrons (such as Cu-3d in the material in focus, LK-99). 
The Hubbard term is given by 

$$E_{DFT+U} = E_{DFT} + \frac{U - J}{2} \Sigma_{\sigma} (Tr [p^{\sigma}]) - Tr[\rho^{\sigma} \rho^{\sigma}],$$
                                                                               
such that $\rho^{\sigma}$ represents the density matrix for the spin, $\sigma$. 

The PBEsol function is an updated Perdew-Burke-Ernzerhof (PBE) functional relevant for improve lattice parameter predictions for solids. 

Quantum ESPRESSO is a software suite that implements density functional theory, resolving the Kohn-Sham equations with plane-wave basis sets, as well as pseudo-
potentials. 


### Application to LK-99 for Superconductivity

To continue with the evaluation of the LK-99 material for the superconductivity property, one must move on to perform phonon calculations, such as electron-phonon
coupling strength, given by $\lambda$, and the critical temperature, $T_c$, using the McMillan-Allen-Dynes formula. The Wannier functions can be used to construct
tight-binding models to analyze the material further. 


: 

In [ ]:
import os
    from pymatgen.core import Structure
    from pymatgen.io.quantumespresso import PWInput
    from IPython.display import FileLink, display

    # loads the relaxed structure

    relaxed_structure = Structure.from_file("relax_pbesol.out")

    # defines the Hubbard U parameters (Cu-3d)

    hubbard_u = {"Cu": {"L": 2, "U": 5.0, "J": 0.0, "Hubbard_U": 5.0}}

    # generates QE input with +U correction

    input_hubbard = PWInput

    (
        
        relaxed_struct, pseudopotentials = 
    
    {
        "Pb": "Pb.pbesol-n-rrkjus_psl.1.0.0.UPF",
        "Cu": "Cu.pbesol-n-rrkjus_psl.1.0.0.UPF",
        "P": "P.pbesol-n-rrkjus_psl.1.0.0.UPF",
        "O": "O.pbesol-n-rrkjus_psl.1.0.0.UPF",
    },

    control = {"calculation": "scf", "tstress": True, "prefix": "pwscf"},

    system = 
    
    {
        "ecutwfc": 60,
        "ecutrho": 480,
        "occupations": "smearing",
        "smearing": "mv",
        "degauss": 0.02,
        "lda_plus_u": True,  # Enable +U
        "hubbard_u": hubbard_u["Cu"]["Hubbard_U"],
        "hubbard_j0": hubbard_u["Cu"]["J"],
    },

    kpts=(4, 4, 4),
    
    )

    input_hubbard.write_file("hubbard_u.in")

    # runs QE with +U

    !mpirun -np 4 pw.x -in hubbard_u.in > hubbard_u.out

    # verifies completion & analyzeS results

    if "JOB COMPLETED" in open("hubbard_u.out").read():

        print("+U calculation succeeded!")
        display(FileLink("hubbard_u.out"))  
    
    # parses total energy

    from pymatgen.io.quantumespresso import QEOutput

    qe_out = QEOutput("hubbard_u.out")

    print(f"Total energy with +U: {qe_out.final_energy} eV")

    else:

        print("+U calculation failed. Check hubbard_u.out.")